### Lab 2: Hyponyms and Hypernyms

In [1]:
import pandas as pd
import wikipedia
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
DATA_PATH_LIST = ['.']
EMBEDDING_MODEL_FILENAME = "wiki_node2vec.bin"
DATA_PATH="/".join(DATA_PATH_LIST+["training_nouns.tsv"])
df = pd.read_csv(DATA_PATH,sep='\t')
%matplotlib inline

4


In [2]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [3]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[3]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[2]))
        print(prestr(val[3]))

### Interactive visualization of hyponyms and hypernyms

In [4]:
button = widgets.Button(description="Draw")
query = widgets.Text(
    value='МАТЬ',
    placeholder='Query',
    description='String:',
    disabled=False
)
display(button,query)




def creategraph(df):
    res = []
    for row in df.values:
        cohyps = row[1].split(",")
        for idx,cohyp in enumerate(cohyps):
            for parent in json.loads(prestr(row[2])):
                res.append((row[0]+'-'+str(idx),parent))
    return res

def graphdraw(b):
    print("graphdraw",query.value)
    subset = df[df['TEXT'].str.contains(query.value.upper())]
    g = nx.DiGraph()
    for el in subset.values:
        cohyps = el[1].split(",")
        print(cohyps)
        syns = idx2syns[el[0]]
        for child in cohyps:
            for parent in json.loads(prestr(el[2])):
                ed = g.add_edge(child,idx2syns[parent],label="is a")
            
    plt.figure(figsize=(15,15))
    pos = nx.nx_agraph.graphviz_layout(g)
    nx.draw(g,with_labels=True,pos=pos)
#     edge_labels=nx.draw_networkx_edge_labels(g,pos=pos)
    plt.show()
button.on_click(graphdraw)

Button(description='Draw', style=ButtonStyle())

Text(value='МАТЬ', description='String:', placeholder='Query')

### Pattern extractor

Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории Natasha.
* https://yargy.readthedocs.io/ru/latest/
* http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
* https://github.com/natasha/natasha

### Токенизатор

In [5]:
from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

['Ростов', '-', 'на', '-', 'Дону']
['Длительностью', '18', 'ч', '.', '10', 'мин', '.']
['Яндекс', '.', 'Такси']
['π', '≈', '3', '.', '1415']
['1', '500', '000', '$']
['http', ':', '/', '/', 'vk', '.', 'com']


# Газеттир
Газеттир нужен для удобной работы с последовательностью слов. Например, можно написать:

In [6]:
from yargy import or_, rule
from yargy.predicates import normalized

RULE = or_(
    rule(normalized('dvd'), '-', normalized('диск')),
    rule(normalized('видео'), normalized('файл'))
)

In [7]:
from yargy import Parser
from yargy.pipelines import morph_pipeline


RULE = morph_pipeline([
    'dvd-диск',
    'видео файл',
    'видеофильм',
    'газета',
    'электронный дневник',
    'эссе',
])

parser = Parser(RULE)
text = 'Видео файл на dvd-диске'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Видео', 'файл']
['dvd', '-', 'диске']


In [8]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


GEO = rule(
    and_(
        gram('ADJF'),  # так помечается прилагательное, остальные пометки описаны в
                       # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
        is_capitalized()
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'федерация',
        'республика'
    })
)


parser = Parser(GEO)
text = '''
В Чеченской республике на день рождения ...
Донецкая народная республика провозгласила ...
Башня Федерация — одна из самых высоких ...
'''
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Чеченской', 'республике']
['Донецкая', 'народная', 'республика']


### Предикаты

Предикат — функция, которая принимает на вход токен и возвращает True или False. В Yargy встроено много готовых предикатов. Полный список есть в справочнике. Предикаты комбинируются с помощью and_, or_ и not_.

In [9]:
from yargy import and_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import is_capitalized, eq


tokenizer = MorphTokenizer()
token = next(tokenizer('Стали'))

predicate = is_capitalized()
print(predicate(token))

predicate = and_(
    is_capitalized(),
    not_(eq('марки'))
)
print(predicate(token))

True
True


### Грамматики
В Yargy используется специальный DSL для описания грамматик. Любую контекстно-свободную грамматику можно описать с помощью конструкций Питона. Например, есть примитивная грамматика для размеров одежды:

KEY -> р. | размер

VALUE -> S | M | L

SIZE -> KEY VALUE



In [10]:
from yargy import rule, or_


KEY = or_(
    rule('р', '.'),
    rule('размер')
).named('KEY')
VALUE = or_(
    rule('S'),
    rule('M'),
    rule('L'),
    rule('XS'),
).named('VALUE')
SIZE = rule(
    KEY,
    VALUE
).named('SIZE')
SIZE.normalized.as_bnf

SIZE -> KEY VALUE
KEY -> 'р' '.' | 'размер'
VALUE -> 'S' | 'M' | 'L' | 'XS'


In [11]:
parser = Parser(
    SIZE
)
text = 'размер M; размер A; размер XS;'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['размер', 'M']
['размер', 'XS']


In [12]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [13]:
START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        eq('-'),
        eq('это')
    ),    
    rule(
        eq('являются')
    ),
    rule(
        eq('является')
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [14]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [15]:
get_hyperonyms("банан")

Банан — название съедобных плодов культивируемых растении рода Банан (Musa); обычно под таковыми понимают Musa acuminata и Musa × paradisiaca, а также Musa balbisiana, Musa fehi, Musa troglodytarum и ряд других. Также бананами могут называть плоды Ensete ventricosum (строго говоря, являющегося представителем другого рода семеиства Банановые). С ботаническои точки зрения банан является ягодои, многосеменнои и толстокожеи. У культурных форм часто отсутствуют семена, ненужные при вегетативном размножении. Плоды имеют длину 6—30 см и диаметр 2—5 см. Соплодия могут состоять из 300 плодов и иметь массу до 50—60 кг.
Бананы — одна из древнеиших пищевых культур, а для тропических стран важнеишее пищевое растение и главная статья экспорта. Спелые бананы широко употребляются в пищу по всему миру, их используют при приготовлении большого количества блюд. Помимо употребления в свежем виде, в кухне некоторых народов бананы могут зажариваться, или вариться как в очищенном, так и в неочищенном виде. И

#### Task 1 (deadline 19.02.2020 24:00)
* Find your name on the spreadsheet https://docs.google.com/spreadsheets/d/1RR2I6toCkebbGU1UK83HS70Ru_l0_o-nnZIHyiFB0No/edit?usp=sharing. In opposite of your name there are 24 words of hyponyms, you have to insert five corresponding hypernyms next to them. Examples of hyponyms and hyperonyms relationship you can find above in the current Jupiter notebook.
* Find for each pair of hyponyms and hypernyms a corresponding snippet of a text with their mentions. The source of the text can be any free resources, e.g., Wikipedia, Google, Yandex, others. You should save the snippets and their URLs within the lab2 folder in your NLP git-repo with .csv file-extension in a single file.

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

In [16]:
import pandas
df = pandas.read_csv('Lab2_Hyperonyms_csv_file.csv', sep='\t')
print(df)

                        WORD;HYPERHONYM;TEXT;REFERENCE
0    ОСОТ;Сорная трава;ОСОТ - Род крупной сорной тр...
1    ОСОТ;Растение;СОТ растение из рода травянистых...
2    ОСОТ;Сорняк;Сорняк, годный на корм;https://пои...
3    ОСОТ;Сорное растение;ОСО́Т, -а, м. Название не...
4    ОСОТ;Трава;СОТ растение из рода травянистых ра...
..                                                 ...
115  ПАНТЮРКИЗМ;Политическое течение;Пантюрки́зм — ...
116  ПАНТЮРКИЗМ;Доктрина;ПАНТЮРКИЗМ — доктрина, воз...
117  ПАНТЮРКИЗМ;Идеалогия;пантюркизм — Идеология, п...
118  ПАНТЮРКИЗМ;Политическое движение;Пантюрки́зм —...
119  ПАНТЮРКИЗМ;Идеологическое движение;Пантюрки́зм...

[120 rows x 1 columns]


In [17]:
df.head()

,WORD;HYPERHONYM;TEXT;REFERENCE
0,ОСОТ;Сорная трава;ОСОТ - Род крупной сорной тр...
1,ОСОТ;Растение;СОТ растение из рода травянистых...
2,"ОСОТ;Сорняк;Сорняк, годный на корм;https://пои..."
3,"ОСОТ;Сорное растение;ОСО́Т, -а, м. Название не..."
4,ОСОТ;Трава;СОТ растение из рода травянистых ра...


In [18]:
import pandas
data = pandas.read_excel('Hyperonyms_csv.xlsx', sheet_name='Table')
print(data)

           WORD               HYPORONIMS  \
0          ОСОТ             Сорная трава   
1          ОСОТ                 Растение   
2          ОСОТ                   Сорняк   
3          ОСОТ          Сорное растение   
4          ОСОТ                    Трава   
..          ...                      ...   
115  ПАНТЮРКИЗМ     Политическое течение   
116  ПАНТЮРКИЗМ                 Доктрина   
117  ПАНТЮРКИЗМ                Идеалогия   
118  ПАНТЮРКИЗМ    Политическое движение   
119  ПАНТЮРКИЗМ  Идеологическое движение   

                                                  TEXT  \
0    Осот — это род крупной сорной травы из семейст...   
1    Осот — это растение из рода травянистых растен...   
2                   Осот — это сорняк, годный на корм.   
3    Осот — это название нескольких видов сорных тр...   
4    Осот — это трава, проявляющая свои лечебные св...   
..                                                 ...   
115  Пантюркизм — это политическое течение, распрос...   
116  Па

In [19]:
data.head()

,WORD,HYPORONIMS,TEXT,REFERENCE
0,ОСОТ,Сорная трава,Осот — это род крупной сорной травы из семейст...,https://kartaslov.ru/значение-слова/осот
1,ОСОТ,Растение,Осот — это растение из рода травянистых растен...,https://dic.academic.ru/dic.nsf/enc2p/292478
2,ОСОТ,Сорняк,"Осот — это сорняк, годный на корм.",https://поискслов.рф/wd/осот
3,ОСОТ,Сорное растение,Осот — это название нескольких видов сорных тр...,https://поискслов.рф/wd/осот
4,ОСОТ,Трава,"Осот — это трава, проявляющая свои лечебные св...",https://agronomu.com/bok/2210-chem-polezen-oso...


In [20]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text, end='\n\n')
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [21]:
i = 0
j = 0
m = 0
s = 0
e = 5
s2 = 0
e2 = 5
p = 0
for row in range(len(data)):
    word = data['WORD'][i]
    print('Гиперонимы для: ', word, end='\n\n')
    i += 5
    get_hyperonyms(word)
    print('\n' + 'Найденные гиперонимы: ', end = '')
    for j in range(s,e):
        hyporonim = data['HYPORONIMS'][j]
        s += 1
        e += 1
        print(hyporonim, end = ' ')
    for m in range(s2,e2):
        proof = data['TEXT'][m]
        s2 += 1
        e2 += 1
        print('\n\n' + str(p+1)+ '. ' + proof, end = '\n')
        p += 1
    p = 0
    print('\n\n\n')


Гиперонимы для:  ОСОТ

Осот (лат. Sonchus) — род травянистых растении семеиства Астровые или Сложноцветные (Asteraceae). Однолетние, двулетние или многолетние травы, иногда древеснеющие у основания. Род включает около 130 видов. Некоторые виды, например Осот огородныи (Sonchus oleraceus) и Осот полевои (Sonchus arvensis) являются злостными сорными растениями.
Иногда осотом называют некоторые виды бодяка (Cirsium).


Найденные гиперонимы: Сорная трава Растение Сорняк Сорное растение Трава 

1. Осот — это род крупной сорной травы из семейства сложноцветных.


2. Осот — это растение из рода травянистых растений семейства Астровые или Сложноцветные


3. Осот — это сорняк, годный на корм.


4. Осот — это название нескольких видов сорных травянистых растений сем. сложноцветных.


5. Осот — это трава, проявляющая свои лечебные свойства благодаря наличию витаминов, алкалоидов, холина и др. полезных веществ.




Гиперонимы для:  ОСТЕОМИЕЛИТ

Остеомиелит (от др.-греч. οστεον — «кость»; μυελος — 


Найденные гиперонимы: Расслабление Развлечение Открепление Освобождение от привязи Отвязывание 

1. Отвязка — это отдых, развлечение, расслабление, празднество, отпуск.


2. Отвязка — это отдых, развлечение, расслабление, празднество, отпуск.


3. Отвязка — это действие, означающее открепление.


4. Отвязка — это освобождение от привязи.


5. Отвязка — это действие, которое означает "отвязывать", "отвязывание".




Гиперонимы для:  ОТКАЗОУСТОЙЧИВОСТЬ

Отказоустоичивость (англ. Fault tolerance) — своиство техническои системы сохранять свою работоспособность после отказа одного или нескольких составных компонентов.
Отказоустоичивость определяется количеством любых последовательных единичных отказов компонентов, после которого сохраняется работоспособность системы в целом. Базовыи уровень отказоустоичивости подразумевает защиту от отказа одного любого элемента — исключение единои точки отказа.
Основнои способ повышения отказоустоичивости — избыточность. Наиболее эффективныи метод избыточ

Падеграс, или па-де-грас (от фр. pas — шаг и фр. grace — грация, изящество) — русскии парныи бальныи танец, для которого характерно чередование мягких шагов с приседаниями и фиксированных поз.


Найденные гиперонимы: Парный бальный танец Музыкальное произведение Русский танец Музыка Стиль танца 

1. Падеграс — это парный бальный танец установленной композиции, со спокойными движениями, в котором медленные шаги чередуются с приседаниями и фиксированными позами.


2. Падеграс — это музыкальное произведение в ритме такого танца.


3. Падеграс — это парный бальный танец установленной композиции, со спокойными движениями, в котором медленные шаги чередуются с приседаниями и фиксированными позами.


4. Падеграс — это музыка к такому танцу. 


5. Падеграс — это стиль парного танца установленной композиции, со спокойными движениями, в котором медленные шаги чередуются с приседаниями и фиксированными позами.




Гиперонимы для:  ПАДУЧАЯ

Эпилепсия (лат. epilepsia от др.-греч. επιληψια ← επιληπτ

Пальпация (лат. palpatio — «ощупывание») — физическии метод медицинскои диагностики, проводимыи путем ощупывания тела пациента. Как способ изучения своиств пульса, пальпация упоминается еще в трудах Гиппократа. В качестве метода исследования внутренних органов, пальпация получила широкое распространение в Европе лишь со второи половины XIX века после работ Р. Лаэннека, И. Шкоды, В. П. Образцова и др.
Известныи клиническии афоризм: «Стетоскоп уж тем хорош, что заставляет врача хотя бы на 15 см приблизиться к больному, а пальпация и вовсе обеспечивает контакт врача с больным».


Найденные гиперонимы: Врачебный метод Метод медицинской диагностики Ощупывание Прощупывание Физический метод обследования 

1. Пальпация — это врачебный метод исследования больного.


2. Пальпация — это метод медицинской диагностики, проводимый путём ощупывания тела пациента.


3. Пальпация — это ощупывание больного.


4. Пальпация — это прощупывание больного.


5. Пальпация — это физический метод медицинской диа

KeyError: 120

In [22]:
def get_hyper(row):
    main_word = row['WORD']
    text = row['TEXT']
    
    HYPONYM = eq(utils.deaccent(main_word.lower()))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, END),
        rule(END, MID, END),
        rule(END, MID, MID, END),
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    # print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps

In [23]:
data['dec_hyper'] = data.apply(lambda row: get_hyper(row), axis=1)

ОСОТ [['осот', '—', 'это', 'род', 'крупнои', 'сорнои', 'травы', 'из', 'семеиства']]
ОСОТ [['осот', '—', 'это', 'растение', 'из', 'рода', 'травянистых', 'растении', 'семеиства']]
ОСОТ [['осот', '—', 'это', 'сорняк']]
ОСОТ [['осот', '—', 'это', 'название']]
ОСОТ [['осот', '—', 'это', 'трава']]
ОСТЕОМИЕЛИТ [['это', 'гноино', '—', 'некротическии', 'процесс']]
ОСТЕОМИЕЛИТ [['остеомиелит', '—', 'это', 'гноиное', 'воспаление']]
ОСТЕОМИЕЛИТ [['остеомиелит', '—', 'это', 'инфекционныи', 'воспалительныи', 'процесс']]
ОСТЕОМИЕЛИТ [['остеомиелит', '—', 'это', 'воспаление', 'костного', 'мозга']]
ОСТЕОМИЕЛИТ [['остеомиелит', '—', 'это', 'инфекционное', 'поражение']]
ОСТРАСТКА [['острастка', '—', 'это', 'строгое', 'предупреждение']]
ОСТРАСТКА [['острастка', '—', 'это', 'угроза']]
ОСТРАСТКА [['острастка', '—', 'это', 'угроза']]
ОСТРАСТКА [['острастка', '—', 'это', 'угроза']]
ОСТРАСТКА [['острастка', '—', 'строгое', 'предупреждение']]
ОСТРИЦА [['острица', '—', 'это', 'мелкии', 'глист']]
ОСТРИЦА [['остри